# [Actual Causation](https://pyphi.readthedocs.io/en/latest/examples/actual_causation.html)
*This attempts to do ths same calculations as the rtd paper (link above) but use new classes for Network, TPM (etc.).*  

In [1]:
import pyphi
from pyphi import actual, config, Direction

import numpy as np
import pandas as pd

import pyphi.TpmClass as tc  # Prototype code

## [Configuration](https://pyphi.readthedocs.io/en/latest/examples/actual_causation.html#configuration)

In [2]:
>>> config.PARTITION_TYPE = 'TRI'
>>> config.PICK_SMALLEST_PURVIEW = True
>>> config.VALIDATE_SUBSYSTEM_STATES = False

## [Computation](https://pyphi.readthedocs.io/en/latest/examples/actual_causation.html#computation)

In [3]:
network = pyphi.examples.actual_causation()
network.tpm

array([[[0., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 1.]]])

In [4]:
network.cm

array([[1, 1],
       [1, 1]])

In [5]:
# New style
net2 = tc.Network(network)
network2 = net2.legacy()
net2._tpdf

(0, 0)  (0, 1)  (1, 0)  (1, 1)
(0, 0)     1.0     0.0     0.0     0.0
(0, 1)     0.0     1.0     0.0     0.0
(1, 0)     0.0     1.0     0.0     0.0
(1, 1)     0.0     0.0     0.0     1.0

In [6]:
network2.tpm

array([[[0., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 1.]]])

In [7]:
network2.cm

array([[1, 1],
       [1, 1]])

In [8]:
# New style
net2._ncdf

OR  AND
OR    1    1
AND   1    1

In [9]:
pyphi.convert.sbs2sbn(net2._tpdf.to_numpy())

array([[[0., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 1.]]])

In [10]:
pyphi.convert.sbn2sbs(network.tpm).shape

(4, 4)

In [11]:
pyphi.convert.sbs2sbn(network.tpm).shape

(2, 1)

In [12]:
>>> pyphi.convert.state_by_node2state_by_state(network.tpm)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [13]:
>>> OR = 0
>>> AND = 1

In [14]:
>>> X = Y = (OR, AND)
>>> X_state = Y_state = (1, 0)

In [15]:
X

(0, 1)

In [16]:
X_state

(1, 0)

In [17]:
transition = actual.Transition(network, X_state, Y_state, X, Y)
transition

Transition([OR, AND] ━━▶ [OR, AND])

In [18]:
# from New style
actual.Transition(network2, X_state, Y_state, X, Y)

Transition([OR, AND] ━━▶ [OR, AND])

In [19]:
transition.effect_repertoire((OR,), (OR, AND))

array([[0. , 0. ],
       [0.5, 0.5]])

In [20]:
transition.cause_repertoire((OR, AND), (OR,))

array([[0.5],
       [0.5]])

In [21]:
transition.effect_ratio((OR,), (OR,))

0.415037

In [22]:
>>> transition.effect_ratio((OR,), (AND,))

-0.584963

In [23]:
>>> transition.cause_ratio((OR,), (OR, AND))

0.415037

In [24]:
link = transition.find_mip(Direction.EFFECT, (OR, AND), (OR, AND))

In [25]:
link.alpha

0.0

In [26]:
link.partition

 ∅    OR    AND
─── ✕ ─── ✕ ───
 ∅    OR    AND

In [27]:
>>> link = transition.find_mip(Direction.CAUSE, (OR, AND), (OR, AND))
>>> link.alpha

0.169925

In [28]:
transition.find_actual_cause((OR, AND))

CausalLink
  α = 0.1699  [OR, AND] ◀━━ [OR, AND]

## [Accounts](https://pyphi.readthedocs.io/en/latest/examples/actual_causation.html#accounts)

In [29]:
>>> account = actual.account(transition)
>>> print(account)  


      Account (5 causal links)     
***********************************
Irreducible effects
α = 0.415  [OR] ━━▶ [OR]
α = 0.415  [AND] ━━▶ [AND]
Irreducible causes
α = 0.415  [OR] ◀━━ [OR]
α = 0.415  [AND] ◀━━ [AND]
α = 0.1699  [OR, AND] ◀━━ [OR, AND]


In [30]:
>>> len(account)

5

## [Irreducible Accounts](https://pyphi.readthedocs.io/en/latest/examples/actual_causation.html#irreducible-accounts)

In [31]:
>>> sia = actual.sia(transition)
>>> sia.alpha

0.169925

In [32]:
sia.partitioned_account


 Account (4 causal links) 
**************************
Irreducible effects
α = 0.415  [OR] ━━▶ [OR]
α = 0.415  [AND] ━━▶ [AND]
Irreducible causes
α = 0.415  [OR] ◀━━ [OR]
α = 0.415  [AND] ◀━━ [AND]

In [33]:
sia.cut

KCut CAUSE
 ∅    OR    AND
─── ✕ ─── ✕ ───
 ∅    OR    AND

### The following produces error if run again after full notebook is run once.

In [34]:
all_accounts = actual.nexus(network, X_state, Y_state)

In [35]:
>>> for n in all_accounts:
...     print(n.transition, n.alpha)


Transition([OR] ━━▶ [OR]) 2.0
Transition([AND] ━━▶ [AND]) 2.0
Transition([OR, AND] ━━▶ [OR, AND]) 0.169925


In [36]:
>>> cn = actual.causal_nexus(network, X_state, Y_state)
>>> print(cn.alpha)
>>> cn.transition

2.0


Transition([OR] ━━▶ [OR])

## [Disjunction of conjunctions](https://pyphi.readthedocs.io/en/latest/examples/actual_causation.html#disjunction-of-conjunctions)
If you are interested in exploring further, the disjunction of conjunctions network from Figure 9 is provided as well:

In [37]:
>>> network = pyphi.examples.disjunction_conjunction_network()
>>> cn = actual.causal_nexus(network, (1, 0, 1, 0), (0, 0, 0, 1))

In [38]:
>>> print(cn.transition)
>>> cn.alpha
2.0

Transition([C] ━━▶ [D])


2.0